# 🚀 Mega ToT Reasoning Labs — Part II  
**15‑Puzzle, N‑Queens, Knight’s Tour (+ optional LLM guidance)**

Continues the Day 5 exploration with *three* fresh puzzles that benefit from branching search:

1. **15‑Puzzle** (4 × 4 slider)  
2. **N‑Queens** (8‑Queens default, N configurable)  
3. **Knight’s Tour** (closed tour on 5 × 5 or 8 × 8)  

Each demo offers:

* Plain heuristic scoring  
* **LLM‑scored** option (set `OPENAI_API_KEY`)  
* Experiments & visual helpers

## 🔧 0. Setup

In [ ]:
%pip -q install --upgrade openai networkx matplotlib
import os, random, copy, math, itertools, collections, time
import openai, networkx as nx, matplotlib.pyplot as plt
openai.api_key = os.getenv("OPENAI_API_KEY")
MODEL="gpt-4o-mini"

### Generic Tree‑of‑Thought Beam Solver

In [ ]:
class ToTSolver:
    def __init__(self, expand, score, beam=10, max_depth=100, verbose=False):
        self.expand=expand; self.score=score
        self.beam=beam; self.max_depth=max_depth; self.verbose=verbose
    def solve(self, start, is_goal):
        frontier=[(start, self.score(start))]
        for depth in range(self.max_depth):
            if self.verbose: print("Depth", depth, "frontier", len(frontier))
            new=[]
            for state,_ in frontier:
                if is_goal(state): return state
                for child in self.expand(state):
                    new.append((child, self.score(child)))
            frontier=sorted(new,key=lambda x:-x[1])[:self.beam]
        return frontier[0][0]

---

## 1️⃣ 15‑Puzzle (4×4 Sliding Tiles)

In [ ]:
GOAL15=tuple(range(1,16))+ (0,)

def idx_rc(idx, size=4): return divmod(idx,size)

def expand15(state):
    size=4
    idx0=state.index(0)
    r,c=idx_rc(idx0,size)
    children=[]
    for dr,dc in [(-1,0),(1,0),(0,-1),(0,1)]:
        nr,nc=r+dr,c+dc
        if 0<=nr<size and 0<=nc<size:
            nidx=nr*size+nc
            lst=list(state)
            lst[idx0],lst[nidx]=lst[nidx],lst[idx0]
            children.append(tuple(lst))
    return children

def manhattan15(state):
    size=4
    dist=0
    for i,val in enumerate(state):
        if val==0: continue
        gr,gc=idx_rc(val-1,size)
        cr,cc=idx_rc(i,size)
        dist+=abs(gr-cr)+abs(gc-cc)
    return -dist

def llm_score15(state):
    if not openai.api_key: return manhattan15(state)
    prompt=f"15‑puzzle board {state}. Score -40 (worst) to 0 (solved) by how close it is to goal order. Single integer only."
    try:
        rep=openai.ChatCompletion.create(model=MODEL,messages=[{"role":"user","content":prompt}],temperature=0)
        return int(rep.choices[0].message.content.strip())
    except: return manhattan15(state)

start15=(1,2,3,4,5,6,7,8,9,10,11,12,0,13,14,15)
solver15=ToTSolver(expand15,llm_score15,beam=30,max_depth=60)
sol15=solver15.solve(start15, lambda s:s==GOAL15)
print("Solved?", sol15==GOAL15)

#### 📝 Experiments

* Shuffle `start15` harder; watch beam width effect  
* Compare heuristic vs. LLM score speed & success

---

## 2️⃣ N‑Queens (default N=8)

In [ ]:
N=8
def expand_q(state):
    row=len(state)
    if row==N: return []
    children=[]
    for col in range(N):
        if all(col!=c and abs(col-c)!=row-i for i,c in enumerate(state)):
            children.append(state+(col,))
    return children

def score_q(state):
    return len(state)

def llm_score_q(state):
    if not openai.api_key: return score_q(state)
    prompt=f"We are placing queens. Board size {N}. Current rows: {state}. Rate 0‑{N} how promising this partial placement is toward no‑conflict full solution."
    try:
        out=openai.ChatCompletion.create(model=MODEL,messages=[{"role":"user","content":prompt}],temperature=0)
        return float(out.choices[0].message.content.strip())
    except: return score_q(state)

solverQ=ToTSolver(expand_q,llm_score_q,beam=30,max_depth=N)
solutionQ=solverQ.solve(tuple(), lambda s: len(s)==N)
print("Solution cols per row:", solutionQ)

#### 📝 Play

* Try N=10, N=12 with larger beam  
* Does LLM pruning help beyond simple `len(state)` score?

---

## 3️⃣ Knight’s Tour (5×5 default)

Find a path that visits every square exactly once.

In [ ]:
SIZE=5
kn_moves=[(2,1),(1,2),(-1,2),(-2,1),(-2,-1),(-1,-2),(1,-2),(2,-1)]
def valid(r,c): return 0<=r<SIZE and 0<=c<SIZE
def expand_knight(path):
    r,c=path[-1]
    children=[]
    for dr,dc in kn_moves:
        nr,nc=r+dr,c+dc
        if valid(nr,nc) and (nr,nc) not in path:
            children.append(path+[(nr,nc)])
    return children

def score_knight(path):
    return len(path)

def llm_score_kn(path):
    if not openai.api_key: return score_knight(path)
    prompt=f"Knight path length {len(path)}/{SIZE*SIZE}. Rate promise 0‑{SIZE*SIZE}."
    try:
        rep=openai.ChatCompletion.create(model=MODEL,messages=[{"role":"user","content":prompt}],temperature=0)
        return float(rep.choices[0].message.content.strip())
    except: return score_knight(path)

start_pos=(0,0)
solverK=ToTSolver(expand_knight,llm_score_kn,beam=50,max_depth=SIZE*SIZE)
solK=solverK.solve([start_pos], lambda p: len(p)==SIZE*SIZE)
print("Tour found length:", len(solK))

### Visualize (optional)

In [ ]:
if len(solK)==SIZE*SIZE:
    G=nx.DiGraph()
    for i in range(len(solK)-1):
        G.add_edge(solK[i], solK[i+1])
    pos={(r,c):(c,-r) for r,c in G.nodes}
    plt.figure(figsize=(6,6))
    nx.draw(G,pos,with_labels=True,node_size=200,node_color='lightblue',arrows=False)
    plt.title("Knight’s Tour path")
    plt.show()

---

## 🔗 References

* Atkinson, “15‑Puzzle Optimal Solutions Revisited”, 2016  
* Knuth, “Dancing Links” (N‑Queens)  
* M. Conrad, “Knight’s Tour Algorithms”, 2022